In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：使用get_metadata自定义训练表格回归模型进行在线预测并带有可解释性

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/sdk_custom_tabular_regression_online_explain_get_metadata.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fexplainable_ai%2Fsdk_custom_tabular_regression_online_explain_get_metadata.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/explainable_ai/sdk_custom_tabular_regression_online_explain_get_metadata.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/sdk_custom_tabular_regression_online_explain_get_metadata.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

本教程演示如何使用Vertex AI SDK来训练和部署一个用于在线预测的自定义表格回归模型，并提供解释。

了解更多关于[Vertex可解释人工智能](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview)和[Vertex AI预测](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions)的信息。

### 目标

在本教程中，您将学习如何使用Vertex AI SDK在Google预构建的Docker容器中从Python脚本创建自定义模型。然后，您将学习如何使用Vertex AI预测服务进行带解释的在线预测请求。或者，您可以使用`gcloud`命令行工具或在Cloud Console中在线创建自定义模型。

本教程使用以下Google Cloud ML服务：

- Vertex AI训练
- Vertex AI在线预测
- Vertex可解释AI
- Vertex AI模型资源
- Vertex AI终结点资源

执行的步骤包括：

- 创建一个用于训练TensorFlow模型的Vertex AI定制作业。
- 训练一个TensorFlow模型。
- 检索并加载模型构件。
- 查看经过训练的模型的模型评估。
- 设置解释参数。
- 将模型上传为Vertex AI模型资源。
- 将Model资源部署到一个提供终结点资源。
- 进行带解释的预测。
- 取消部署Model资源。

数据集

本教程使用的数据集是[波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。您在本教程中使用的数据集版本是从TensorFlow SDK中获取的版本。训练好的模型预测房屋的中位价格，单位为1千美元。

成本

本教程使用了Google Cloud的可计费组件：

* Vertex AI
* 云存储

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[云存储价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预计使用量生成成本估算。

开始吧

### 为 Python 安装 Vertex AI SDK 和其他必需的软件包

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 tensorflow==2.15.1

重新启动运行时（仅支持Colab）

为了使用新安装的软件包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核将重新启动。在继续下一步之前，请等到它完成。⚠️</b>
</div>

### 验证您的笔记本环境（仅限Colab）

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

设置Google Cloud项目信息

要开始使用Vertex AI，您必须有一个现有的Google Cloud项目。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶

创建一个存储桶，用于存储诸如数据集之类的中间工件。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

为Python和其他库初始化Vertex SDK

为您的项目和相应的存储桶初始化Vertex AI SDK。

In [ ]:
import os

import numpy as np
import tensorflow as tf
from google.cloud import aiplatform
from google.cloud.aiplatform.explain.metadata.tf.v2 import \
    saved_model_metadata_builder

aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`以使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个支持GPU的容器映像，每个VM分配了4个 Nvidia Telsa T4 GPU，则应指定：

(aiplatform.AcceleratorType.NVIDIA_TESLA_T4, 4)

否则，指定`(None, None)`以使用一个在CPU上运行的容器映像。

在[这里](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)了解更多关于您地区的硬件加速器支持。

**注意**：TF 2.3之前的GPU支持版本将无法在本教程中加载自定义模型。这是一个已知问题，在TF 2.3中已经修复 - 这是由于在服务函数中生成的静态图操作。如果您在自己的自定义模型上遇到此问题，请使用支持GPU的TF 2.3容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 设置预构建容器

设置用于训练和预测的预构建Docker容器映像。


有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。


有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`以配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9GB内存
 - `vCPUs`：数量\[2, 4, 8, 16, 32, 64, 96 \]

**注意**：以下类型不支持用于训练:

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

**注意**：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

## 自定义模型创建和训练

现在你已经准备好开始创建自己的定制模型并进行波士顿房价预测的训练。

### 检查培训包

#### 包布局

在开始培训之前，看一下如何为自定义培训任务组装 Python 包。解压后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到 Docker 镜像的操作环境中的说明。

文件`trainer/task.py`是执行自定义培训任务的 Python 脚本。*注意*，当在工作池规格中引用`trainer/task.py`时，目录斜杠被替换为点号，文件后缀（`.py`）被省略（`trainer.task`）。

#### 包组装

在以下单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 的内容

在下一个单元格中，写出训练脚本 *task.py* 的内容。

总结一下，该脚本执行以下步骤：

- 从 `--model_dir` 命令行参数中检索要保存模型产物的目录。如果未指定，则使用环境变量 `AIP_MODEL_DIR` 中设置的目录。
- 从 TF.Keras 内置数据集中加载波士顿房屋数据集。
- 使用 TF.Keras 模型 API 构建一个简单的深度神经网络模型。
- 编译模型（`compile()`）。
- 根据 `args.distribute` 参数设置训练分发策略。
- 使用由 `args.epochs` 参数指定的周期训练模型（`fit()`）。
- 将训练好的模型保存到指定的模型目录（`save(args.model_dir)`）。
- 将每个特征的最大值保存到指定的参数文件中（`f.write(str(params))`）。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))


def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)


# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
BATCH_SIZE = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_dnn_model()

# Train the model
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=GLOBAL_BATCH_SIZE)
model.save(args.model_dir)

把培训脚本存储到您的云存储桶中。接下来，将培训文件夹打包成压缩的tar球，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_boston.tar.gz

### 创建和运行定制的训练任务

要训练一个定制的模型，需要执行两个步骤：
1）创建一个定制的训练任务
2）指定训练的参数并运行任务。

#### 创建定制的训练任务

使用`CustomTrainingJob`类创建一个定制的训练任务，包含以下参数：

- `display_name`：定制训练任务的人类可读名称。
- `container_uri`：训练容器镜像。
- `requirements`：训练容器镜像的软件包要求（例如，pandas）。
- `script_path`：训练脚本的相对路径。

In [ ]:
job = aiplatform.CustomTrainingJob(
    display_name="boston",
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs==0.7.1", "tensorflow-datasets==4.4"],
)

print(job)

### 准备您的训练参数

现在为您的自定义训练容器定义命令行参数：

- `args`：要传递给作为容器入口点设置的可执行文件的命令行参数。
  - `--model-dir`：用于指定存储模型产物的命令行参数。您可以使用以下两种方法之一指定产物的存储位置。
      - **方法1**（将`DIRECT`设置为`True`）：您将云存储位置作为命令行参数传递给您的训练脚本。
      - **方法2**（将`DIRECT`设置为`False`）：服务将云存储位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本。在这种情况下，您需要在作业规范中告诉服务模型产物的位置。
  - `--epochs`：训练的时代数量。
  - `--steps`：每个时代的步骤数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, "model")

EPOCHS = 20
STEPS = 100

DIRECT = True  # Set False to use AIP_MODEL_DIR
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

运行自定义训练作业

接下来，您可以通过调用`run()`方法来运行自定义作业以启动训练作业，参数如下：

- `args`：要传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数（replica_count = 1 表示单节点训练）。
- `machine_type`：计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作节点的加速器数量。
- `base_output_dir`：用于写入模型工件的 Cloud 存储位置。
- `sync`：设置为**True**以等待作业完成。

In [ ]:
if TRAIN_GPU:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=True,
    )
else:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

model_path_to_deploy = MODEL_DIR

加载保存的模型

您的模型以TensorFlow SavedModel格式存储在Cloud Storage存储桶中。现在，从Cloud Storage存储桶中加载它，然后您就可以运行模型评估并准备进行预测请求。

要加载模型，您可以使用TF.Keras `model.load_model()` 方法，将Cloud Storage路径传递给它，该路径是由`MODEL_DIR`指定的存储模型的位置。

In [ ]:
local_model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

### 加载评估数据

使用`tf.keras.datasets`中的`load_data()`方法，加载波士顿房价测试（留出）数据。这会返回一个包含两个元素的元组。第一个元素是训练数据，第二个是测试数据。每个元素也是一个包含两个元素的元组：特征数据和对应的标签（自住房屋的中位值）。

您不需要训练数据，因此加载为`(_, _)`。

在进行评估之前，您需要对数据进行预处理。通过将每个值除以该列的最大值来对每列中的数据进行归一化（重新缩放）。这将用32位浮点数替换每个单个值，该值介于0和1之间。

In [ ]:
from tensorflow.keras.datasets import boston_housing

(_, _), (x_test, y_test) = boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)


def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float32)
    return feature


# Let's save one data item that has not been scaled
x_test_notscaled = x_test[0:1].copy()

for _ in range(13):
    x_test[_] = scale(x_test[_])
x_test = x_test.astype(np.float32)

print(x_test.shape, x_test.dtype, y_test.shape)
print("scaled", x_test[0])
print("unscaled", x_test_notscaled)

### 执行模型评估

使用模型的`evaluate()`方法进行评估。

In [ ]:
local_model.evaluate(x_test, y_test)

获取服务函数签名

您可以通过重新加载模型到内存中，并查询每个层对应的签名，来获取您模型输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务函数而不是模型，因此，您需要知道服务函数的输入层名称 -- 在后续进行预测请求时会用到。

您还需要知道服务函数的输入和输出层名称，以构建解释元数据 -- 这将在接下来讨论。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)
serving_output = list(loaded.signatures["serving_default"].structured_outputs.keys())[0]
print("Serving function output:", serving_output)

input_name = local_model.input.name
print("Model input name:", input_name)
output_name = local_model.output.name
print("Model output name:", output_name)

### 说明规范

在进行预测时，如果要获取解释，您必须启用解释功能，并在将自定义模型上传到 Vertex AI 模型资源时设置相应的设置。这些设置被称为解释元数据，包括：

- `参数`：这是用于在模型上解释的可解释性算法的规范。您可以在以下选项中选择：
  - Shapley（**注意**：不建议用于图像数据，因为可能涉及耗时的操作）
  - XRAI
  - Integrated Gradients
- `元数据`：这是算法在您自定义模型上应用的规范。

#### 解释参数

详细了解解释算法设置。

#### Shapley

将结果归因给每个特征，并考虑特征的不同排列。该方法提供了确切Shapley值的采样近似。

用例：
  - 表格数据的分类和回归。

参数：

- `path_count`：这是算法处理的特征上的路径数。确切的Shapley值近似值需要M! 条路径，其中M是特征数。对于 CIFAR10 数据集，这将是 784（28*28）。

对于任何非常数的特征数量，这将导致计算成本过高。您可以将特征上的路径数减少到 M * `path_count`。

#### Integrated Gradients

这是一种基于梯度的方法，可以有效计算具有与 Shapley 值相同公理属性的特征归因。

用例：
  - 表格数据的分类和回归。
  - 图像数据的分类。

参数：

- `step_count`：这是近似剩余总和的步数。步数越多，积分近似越准确。一般的经验法则是 50 步，但随着步数增加，计算时间也会增加。

#### XRAI

基于综合梯度方法，XRAI评估图像的重叠区域，制作出突显相关区域而非像素的显著性地图。

用例：

  - 图像数据的分类。

参数：

- `step_count`：这是近似剩余总和的步数。步数越多，积分近似越准确。一般的经验法则是 50 步，但随着步数增加，计算时间也会增加。

在下一个代码单元格中，将变量`XAI`设置为您在自定义模型上使用的解释性算法。

In [ ]:
XAI = "ig"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aiplatform.explain.ExplanationParameters(PARAMETERS)

#### 解释元数据

解释元数据包括：

- `outputs`：输出的标量值，用于说明要解释的内容。
- `inputs`：用于归因的特征，即它们对输出的贡献。
  
您可以自定义元数据 -- 要解释和要归因的内容，也可以使用方法 `get_metadata_protobuf()` 自动生成元数据。该方法将为解释所有输出和归因所有输入构建元数据。

In [ ]:
builder = saved_model_metadata_builder.SavedModelMetadataBuilder(MODEL_DIR)
metadata = builder.get_metadata_protobuf()
print(metadata)

## 上传模型

接下来，使用 `Model.upload()` 方法将您的模型上传到 Vertex AI 模型注册表，需要以下参数：

- `display_name`：模型资源的人类可读名称。
- `artifact`：训练模型工件的云存储位置。
- `serving_container_image_uri`：用于服务的容器镜像。
- `sync`：指定是异步还是同步执行上传。
- `explanation_parameters`：配置模型预测解释的参数。
- `explanation_metadata`：描述用于解释的模型输入和输出的元数据。

如果 `upload()` 方法以异步方式运行，则可以随后使用 `wait()` 方法阻塞直到完成。

In [ ]:
model = aiplatform.Model.upload(
    display_name="boston",
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    explanation_parameters=parameters,
    explanation_metadata=metadata,
    sync=False,
)

model.wait()

## 部署模型

接下来，部署您的模型以进行在线预测。要部署模型，请调用`deploy()`方法，使用以下参数：

- `deployed_model_display_name`：部署模型的可读名称。
- `traffic_split`：在端点上发送到此模型的流量百分比，指定为一个或多个键/值对的字典。
如果只有一个模型，则指定为 { "0": 100 }，其中 "0" 指的是上传的这个模型，100 表示100%的流量。
如果有现有模型在端点上，流量被分割到多个模型，则使用model_id指定为 { "0": 百分比, model_id: 百分比, ... }，其中model_id是已部署到端点的现有模型的模型ID。百分比必须相加为100。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作人员副本的加速器数量。
- `starting_replica_count`：初始提供的计算实例数量。
- `max_replica_count`：可扩展到的最大计算实例数量。在本教程中，仅提供一个实例。

In [ ]:
DEPLOYED_NAME = "boston"

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

if DEPLOY_GPU:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU,
        accelerator_count=DEPLOY_NGPU,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU,
        accelerator_count=0,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

获取测试项目

从数据集的测试（保留）部分中选择一个示例作为测试项目。

In [ ]:
test_item = x_test[0]
test_label = y_test[0]
print(test_item.shape)

### 使用解释生成预测

现在您的模型资源已部署到端点资源，您可以通过向端点资源发送预测请求来获得在线解释。

#### 请求

每个实例的格式为：

    [feature_list]

由于`explain()`方法可以接受多个项（实例），请将您的单个测试项发送为一个测试项列表。

#### 响应

从`explain()`调用返回的响应是一个带有以下条目的Python字典：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `predictions`：每个实例的预测。
- `deployed_model_id`：生成预测的部署模型资源的Vertex AI标识符。
- `explanations`：Explainable AI返回的特征归因。

In [ ]:
instances_list = [test_item.tolist()]

prediction = endpoint.explain(instances_list)
print(prediction)

### 理解解释回应

首先，看看您的模型预测结果，并将其与实际值进行比较。

In [ ]:
value = prediction[0][0][0]
print("Predicted Value:", value)

### 检查特征归因

查看此特定示例的特征归因。正值的归因意味着特定特征使您的模型预测增加了该数量，负值的归因则相反。

In [ ]:
from tabulate import tabulate

feature_names = [
    "crim",
    "zn",
    "indus",
    "chas",
    "nox",
    "rm",
    "age",
    "dis",
    "rad",
    "tax",
    "ptratio",
    "b",
    "lstat",
]
attributions = (
    prediction.explanations[0].attributions[0].feature_attributions[serving_input]
)

rows = []
for i, val in enumerate(feature_names):
    rows.append([val, test_item[i], attributions[i]])
print(tabulate(rows, headers=["Feature name", "Feature value", "Attribution value"]))

检查你的解释和基准线

为了更好地理解你收到的特征归因，你应该将它们与模型的基线进行比较。在大多数情况下，归因值的总和和基线应该非常接近模型对于每个输入的预测值。另外，请注意，对于回归模型，从解释中返回的 `baseline_score` 对于发送到你的模型的每个示例保持不变。对于分类模型，每个类别都有自己的基线。

在这个部分，发送10个测试示例给你的模型进行预测，以便将特征归因与基线进行比较。然后，通过 `sanity_check_explanations()` 函数对每个测试示例的归因进行一次检查。

获取解释

In [ ]:
# Prepare 10 test examples to your model for prediction
instances = []
for i in range(10):
    instances.append(x_test[i].tolist())

response = endpoint.explain(instances)

#### 合理性检查

在下面的功能中，您对解释进行了合理性检查。

In [ ]:
def sanity_check_explanations(
    explanation, prediction, mean_tgt_value=None, variance_tgt_value=None
):
    passed_test = 0
    total_test = 1
    # `attributions` is a dict where keys are the feature names
    # and values are the feature attributions for each feature
    baseline_score = explanation.attributions[0].baseline_output_value
    print("baseline:", baseline_score)

    # Sanity check 1
    # The prediction at the input is equal to that at the baseline.
    #  Please use a different baseline. Some suggestions are: random input, training
    #  set mean.
    if abs(prediction - baseline_score) <= 0.05:
        print("Warning: example score and baseline score are too close.")
        print("You might not get attributions.")
    else:
        passed_test += 1
        print("Sanity Check 1: Passed")

    print(passed_test, " out of ", total_test, " sanity checks passed.")


i = 0
for explanation in response.explanations:
    try:
        prediction = np.max(response.predictions[i]["scores"])
    except TypeError:
        prediction = np.max(response.predictions[i])
    sanity_check_explanations(explanation, prediction)
    i += 1

清理工作

要清理掉此项目中使用的所有谷歌云资源，您可以[删除用于本教程的谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Undeploy the model from the endpoint
endpoint.undeploy_all()

# Delete the model endpoint
endpoint.delete()

# Delete the model resource
model.delete()

# Delete the custom training job
job.delete()

# Delete the locally generated files
! rm -rf custom custom.tar.gz

delete_bucket = False  # Set True for deletion
if delete_bucket:
    ! gsutil rm -r $BUCKET_URI